## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Note: Directions wanted this ipynb file in sub folder "Vaction search", but I couldn't figure out how to get access to 
#config file in parent folder, so I moved it and didn't run into issues importing g-key. 

# Trouble shoot this later 


# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Severodvinsk,RU,64.5635,39.8302,6.91,76,11,14.58,few clouds
1,1,Jamestown,US,42.0970,-79.2353,18.72,53,0,5.99,clear sky
2,2,Nanchang,CN,28.6833,115.8833,47.46,59,3,3.02,clear sky
3,3,Burns Lake,CA,54.2331,-125.7533,29.62,99,100,3.51,overcast clouds
4,4,Hobart,AU,-42.8794,147.3294,69.03,70,75,1.01,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Busselton,AU,-33.6500,115.3333,83.34,37,0,4.72,clear sky
15,15,Hilo,US,19.7297,-155.0900,80.33,76,20,6.91,few clouds
23,23,Nanakuli,US,21.3906,-158.1547,78.75,60,40,11.50,scattered clouds
41,41,Georgetown,MY,5.4112,100.3354,80.89,73,20,1.01,few clouds
43,43,Bambous Virieux,MU,-20.3428,57.7575,78.22,94,75,11.50,moderate rain
46,46,Weligama,LK,5.9667,80.4167,77.88,83,8,1.14,clear sky
47,47,Avarua,CK,-21.2078,-159.7750,77.05,94,75,8.05,light rain
49,49,Alofi,NU,-19.0595,-169.9187,84.09,74,75,8.05,broken clouds
59,59,Nha Trang,VN,12.2500,109.1833,77.00,76,96,5.59,overcast clouds
60,60,Butaritari,KI,3.0707,172.7902,81.86,80,87,14.83,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                178
City                   178
Country                177
Lat                    178
Lng                    178
Max Temp               178
Humidity               178
Cloudiness             178
Wind Speed             178
Current Description    178
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
preferred_cities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Busselton,AU,-33.6500,115.3333,83.34,37,0,4.72,clear sky
15,15,Hilo,US,19.7297,-155.0900,80.33,76,20,6.91,few clouds
23,23,Nanakuli,US,21.3906,-158.1547,78.75,60,40,11.50,scattered clouds
41,41,Georgetown,MY,5.4112,100.3354,80.89,73,20,1.01,few clouds
43,43,Bambous Virieux,MU,-20.3428,57.7575,78.22,94,75,11.50,moderate rain
...,...,...,...,...,...,...,...,...,...,...
660,660,Maues,BR,-3.3836,-57.7186,75.61,84,98,1.36,overcast clouds
664,664,Brazzaville,CG,-4.2658,15.2832,80.91,89,40,6.91,thunderstorm
668,668,Kaeo,NZ,-35.1000,173.7833,76.98,56,3,8.46,clear sky
671,671,Imbituba,BR,-28.2400,-48.6703,75.09,91,46,3.18,scattered clouds


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Busselton,AU,83.34,clear sky,-33.6500,115.3333,
15,Hilo,US,80.33,few clouds,19.7297,-155.0900,
23,Nanakuli,US,78.75,scattered clouds,21.3906,-158.1547,
41,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
43,Bambous Virieux,MU,78.22,moderate rain,-20.3428,57.7575,
46,Weligama,LK,77.88,clear sky,5.9667,80.4167,
47,Avarua,CK,77.05,light rain,-21.2078,-159.7750,
49,Alofi,NU,84.09,broken clouds,-19.0595,-169.9187,
59,Nha Trang,VN,77.00,overcast clouds,12.2500,109.1833,
60,Butaritari,KI,81.86,overcast clouds,3.0707,172.7902,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # add location to parameters
    params["location"] = f"{lat},{lng}"


    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    #print(hotels)
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel not found... skipping.")
print(hotel_df)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
                City Country  Max Temp Current Description      Lat       Lng  \
8          Busselton      AU     83.34           clear sky -33.6500  115.3333   
15              Hilo      US     80.33          few clouds  19.7297 -155.0900   
23          Nanakuli      US     78.75    scattered clouds  21.3906 -158.1547   
41        Georgetown      MY     80.89          few clouds   5.4112  100.3354   
43   Bambous Virieux      MU     78.22       moderate rain -20.3428   57.7575   
..               ...     ...       ...                 ...      ...       ...   
664      Brazzaville      CG     80.91        thunderstorm  -4.2658   15.2832   
668             Kaeo      NZ     76.98           clear sky -35

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Busselton,AU,83.34,clear sky,-33.6500,115.3333,Observatory Guest House
15,Hilo,US,80.33,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Nanakuli,US,78.75,scattered clouds,21.3906,-158.1547,Camp Pālehua
41,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
43,Bambous Virieux,MU,78.22,moderate rain,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...,...
660,Maues,BR,75.61,overcast clouds,-3.3836,-57.7186,Hotel Miramar
664,Brazzaville,CG,80.91,thunderstorm,-4.2658,15.2832,Hotel Muesse
668,Kaeo,NZ,76.98,clear sky,-35.1000,173.7833,Bushido Lodge
671,Imbituba,BR,75.09,scattered clouds,-28.2400,-48.6703,Praia Hotel Imbituba


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
# marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

gmaps.configure(api_key = g_key)
locations = clean_hotel_df[['Lat','Lng']]
info_box_template='''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>'''

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows() ]
fig = gmaps.figure(center = (30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig
# 11b. Display the figure

Figure(layout=FigureLayout(height='420px'))